In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export mxresnet

Cells will be exported to model_constructor.mxresnet,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# MXResNet.

> MXResNet model.

MXResNet model, disscussed at https://forums.fast.ai/t/how-we-beat-the-5-epoch-imagewoof-leaderboard-score-some-new-techniques-to-consider

In [ ]:
%nbdev_hide
# from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
%nbdev_export
import torch.nn as nn
import sys, torch
from functools import partial
from collections import OrderedDict
from model_constructor.layers import *
from model_constructor.net import Net

# MXResNet constructor.

In [ ]:
mxresnet  = Net(stem_sizes = [3, 32, 64, 64], name='MXResNet', act_fn=Mish())

In [ ]:
mxresnet

MXResNet constructor
 expansion: 1, sa: 0, groups: 1
 stem sizes: [3, 32, 64, 64]
 body sizes [64, 64, 128, 256, 512]

In [ ]:
mxresnet.block_szs, mxresnet.layers

([64, 64, 128, 256, 512], [2, 2, 2, 2])

In [ ]:
%nbdev_collapse_output
mxresnet.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish()
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish()
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): Mish()
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mxresnet.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [ ]:
%nbdev_collapse_output
mxresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish()
    )
    (bl_1): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), str

In [ ]:
%nbdev_collapse_output
mxresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): Mish()
    )
    (bl_1): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): Mish()
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), str

In [ ]:
%nbdev_collapse_output
mxresnet.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

In [ ]:
%nbdev_hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mxresnet()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size"

torch.Size([16, 1000])


In [ ]:
%nbdev_export
%nbdev_export_and_show
mxresnet_parameters = {'stem_sizes': [3, 32, 64, 64], 'act_fn': Mish()}
mxresnet34  = partial(Net, name='MXResnet32', expansion=1, layers=[3, 4,  6, 3], **mxresnet_parameters)
mxresnet50  = partial(Net, name='MXResnet50', expansion=4, layers=[3, 4,  6, 3], **mxresnet_parameters)

In [ ]:
model = mxresnet50(c_out=10)

In [ ]:
model

MXResnet50 constructor
 expansion: 4, sa: 0, groups: 1
 stem sizes: [3, 32, 64, 64]
 body sizes [16, 64, 128, 256, 512]

In [ ]:
model.c_out, model.layers

(10, [3, 4, 6, 3])

# end
model_constructor
by ayasyrev

In [ ]:
%nbdev_hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted 03_xresnet.ipynb.
Converted 04_Net.ipynb.
Converted 05_Twist.ipynb.
Converted 06_YaResNet.ipynb.
Converted index.ipynb.
